Importing a CSV file into a postgres database
Steps:
    1.Import the csv file into a pandas df
    2.clean the table name and remove all extra symbols,spaces, capital letters
    3.clean the column headers and remove all extra symbols,spaces, capital letters
    4.write the create table SQL statement
    5.import the data into the db

In [7]:
import os
import numpy as np
import pandas as pd
import psycopg2

In [10]:
df=pd.read_csv("CO2 Emissions_Canada.csv")
df.head()

,Make,Model,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),Fuel Consumption Comb (mpg),CO2 Emissions(g/km)
0,ACURA,ILX,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33,196
1,ACURA,ILX,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29,221
2,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48,136
3,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25,255
4,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27,244


#clean table name
1. lower case letters
2. remove all white spaces
3.replace",/,\\,# with _

In [11]:
file="CO2 Emissions_Canada"
clean_tbl_name=file.lower().replace(" ","").replace("?","").replace("-","_").replace(r"/","_").replace("%","").replace(")","").replace(r"(","").replace("$","")
clean_tbl_name

'co2emissions_canada'

#clean header name

    lower case letters
    remove all white spaces
    replace",/,\,# with _



In [13]:
#using list comprehension
df.columns=[x.lower().replace(" ","").replace("?","").replace("-","_").replace(r"/","_").replace("%","").replace(")","").replace(r"(","").replace("$","") for x in df.columns]
df.columns

Index(['make', 'model', 'vehicleclass', 'enginesizel', 'cylinders',
       'transmission', 'fueltype', 'fuelconsumptioncityl_100km',
       'fuelconsumptionhwyl_100km', 'fuelconsumptioncombl_100km',
       'fuelconsumptioncombmpg', 'co2emissionsg_km'],
      dtype='object')

In [15]:
df.dtypes

make                           object
model                          object
vehicleclass                   object
enginesizel                   float64
cylinders                       int64
transmission                   object
fueltype                       object
fuelconsumptioncityl_100km    float64
fuelconsumptionhwyl_100km     float64
fuelconsumptioncombl_100km    float64
fuelconsumptioncombmpg          int64
co2emissionsg_km                int64
dtype: object

In [18]:
create table co2emissions_canada
(
make               varchar,
model              varchar,
vehicleclass       varchar,
enginesizel        float,
cylinders          float,
transmission       varchar,
fueltype           varchar,
fuelconsumptioncityl_100km float,
fuelconsumptionhwyl_100km  float,
fuelconsumptioncombl_100km float,
fuelconsumptioncombmpg     float,
co2emissionsg_km           float
);

SyntaxError: invalid syntax (<ipython-input-18-b79b17247eae>, line 1)

In [17]:
#create disctonary to map data types
replacements={
    'object': 'varchar',
    'float64':'float',
    'datetime64':'timestap',
    'timedelta64[ns]':'varchar'
}
replacements

{'object': 'varchar',
 'float64': 'float',
 'datetime64': 'timestap',
 'timedelta64[ns]': 'varchar'}

In [21]:
# , is joined for column and datatype 
col_str=",".join("{} {}".format(n,d)for (n,d)in zip(df.columns,df.dtypes.replace(replacements)))
col_str

'make varchar,model varchar,vehicleclass varchar,enginesizel float,cylinders int64,transmission varchar,fueltype varchar,fuelconsumptioncityl_100km float,fuelconsumptionhwyl_100km float,fuelconsumptioncombl_100km float,fuelconsumptioncombmpg int64,co2emissionsg_km int64'